# Finetuned OPUS - English to French Translation Model

---

## 1. Problem Definition & Objective

### a. Selected project track
**Language Translation System**  

### b. Problem statement
Design an AI model for multilingual translation or transliteration.

### c. Real-world relevance and motivation
High-quality, low-latency translation is essential in real-world applications such as:
- multilingual communication
- cross-border education and content access
- business communication and translation

Most modern translation solutions depend on proprietary cloud APIs. This project demonstrates how a **Transformer-based NMT model can be trained and deployed locally**, making it suitable for privacy-sensitive or offline environments.


In [ ]:
!pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
!pip install -U evaluate sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
import torch
import evaluate
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

## 2. Data Understanding & Preparation

### a. Dataset source
**Public dataset:** Helsinki-NLP/tatoeba  
- Collection of 200k+ sentences and translations   
- The dataset was **downloaded once** using `datasets` library and stored locally for reuse in future runs, without requiring repeated downloads or any external API calls.

### b. Data loading and exploration
The corpus was provided as two aligned files:
- `Tatoeba.en-fr.en`
- `Tatoeba.en-fr.fr`

Data loading involved:
- reading both files line-by-line
- asserting equal line count to preserve alignment
- constructing `Dataset` object in the form: `{"src": English Sentence, "tgt": French Translation}`

### c. Cleaning, preprocessing, feature engineering
Cleaning steps:
- removed null entries
- removed excessively long sentences
- filtered noisy samples (URLs / abnormal text)

Preprocessing steps:
- tokenization using MarianMT SentencePiece tokenizer
- truncation to a max sequence length
- labels created from target sentences for teacher-forcing training

### d. Handling missing values or noise (if applicable)
Missing/noisy rows occurred due to:
- blank sentences
- corrupted strings
- links or non-language tokens

These were handled using rule-based filtering to remove invalid pairs prior to training.

In [ ]:
ds = load_dataset("tatoeba", lang1="en", lang2="fr", trust_remote_code=True)
ds = ds['train']

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

tatoeba.py: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
src_lang = "en"
tgt_lang = "fr"

def src2tgt(example):
  return {
      "src": example["translation"][src_lang],
      "tgt": example["translation"][tgt_lang]
  }

ds = ds.map(src2tgt)
ds

Map:   0%|          | 0/264905 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'translation', 'src', 'tgt'],
    num_rows: 264905
})

In [ ]:
def data_filter(example):
  s, t = example["src"], example["tgt"]

  if s is None or t is None:
    return False
  if len(s.strip()) == 0 or len(t.strip()) == 0:
    return False
  if len(s)>200 or len(t)>200:
    return False
  if s.count("http")>0 or t.count("http")>0:
    return False
  return True

ds = ds.filter(data_filter)
ds

Filter:   0%|          | 0/264905 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'translation', 'src', 'tgt'],
    num_rows: 264375
})

In [ ]:
splits = ds.train_test_split(test_size=0.1)
train_ds = splits["train"]
temp_ds = splits["test"]

temp_splits = temp_ds.train_test_split(test_size=0.5)
val_ds = temp_splits["train"]
test_ds = temp_splits["test"]

train_ds, test_ds, val_ds

(Dataset({
     features: ['id', 'translation', 'src', 'tgt'],
     num_rows: 237937
 }),
 Dataset({
     features: ['id', 'translation', 'src', 'tgt'],
     num_rows: 13219
 }),
 Dataset({
     features: ['id', 'translation', 'src', 'tgt'],
     num_rows: 13219
 }))

## 3. Model / System Design

### a. AI technique used (ML / DL / NLP / LLM / Recommendation / Hybrid)
**Deep Learning (DL) + NLP**  
- Transformer-based **Sequence-to-Sequence Neural Machine Translation**
- Fine-tuning a pretrained MarianMT translation model

### b. Architecture or pipeline explanation
Pipeline:
1. Data loading
2. Cleaning + Filtering
3. Train/Validation/Test split
4. Tokenization
5. Fine-tuning pretrained MarianMT
6. Evaluation (BLEU score)
7. Saving model locally
8. Deployment via Streamlit UI

Model architecture:
- Encoder–Decoder Transformer (Marian NMT)
- Teacher forcing training (cross-entropy loss on decoder outputs)

### c. Justification of design choices
- **MarianMT** chosen because:
  - strong translation baseline
  - lightweight compared to other existing models
  - suitable for fine-tuning with limited compute
- **BLEU metric** chosen as it is widely used for translation benchmarking
- **Streamlit deployment** chosen for a simple offline UI demonstration


In [ ]:
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a0b4de55-adab-409c-ab37-501bffd5f958)')' thrown while requesting HEAD https://huggingface.co/Helsinki-NLP/opus-mt-en-fr/resolve/main/config.json
Retrying in 1s [Retry 1/5].


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
max_length = 128

def preprocess(batch):
  inputs = batch['src']
  targets = batch['tgt']

  model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_length, truncation=True)

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

train_tok = train_ds.map(preprocess, batched=True)
val_tok   = val_ds.map(preprocess, batched=True).select(range(2000))
test_tok  = test_ds.map(preprocess, batched=True)

Map:   0%|          | 0/237937 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5b7555fb-8c31-400b-b9a6-fd900bdea3ac)')' thrown while requesting HEAD https://huggingface.co/Helsinki-NLP/opus-mt-en-fr/resolve/refs%2Fpr%2F9/model.safetensors.index.json
Retrying in 1s [Retry 1/5].


model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

Map:   0%|          | 0/13219 [00:00<?, ? examples/s]

Map:   0%|          | 0/13219 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt_finetuned",
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=2000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    generation_num_beams=2,
    fp16=True,
    logging_dir="./logs",
    save_total_limit=2,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
bleu = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
  preds, labels = eval_preds
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels = [[(x if x != -100 else tokenizer.pad_token_id) for x in label] for label in labels]
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  score = bleu.compute(predictions=decoded_preds, references=[[x] for x in decoded_labels])
  return {"bleu": score["score"]}

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

## 4. Core Implementation

### a. Model training / inference logic
Training workflow implemented using HuggingFace:
- `MarianMTModel`
- `MarianTokenizer`
- `Seq2SeqTrainer`
- `Seq2SeqTrainingArguments`

Inference:
- input text tokenized → moved to GPU/CPU device
- translation generated using `model.generate()`
- decoded to output sentence

### b. Code must run top-to-bottom without errors
Code was structured as a deterministic pipeline:
- fixed train/val/test splits
- model saving includes config + tokenizer + weights for consistent reloading


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
print(trainer.args.device)
trainer.train()

/tmp/ipython-input-2658500581.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


cuda:0


Epoch,Training Loss,Validation Loss,Bleu
1,0.534600,0.509946,53.932683
2,0.464800,0.493752,55.289997
3,0.411800,0.489015,55.447514


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=44616, training_loss=0.4785741220692783, metrics={'train_runtime': 3888.4955, 'train_samples_per_second': 183.57, 'train_steps_per_second': 11.474, 'total_flos': 3490713524699136.0, 'train_loss': 0.4785741220692783, 'epoch': 3.0})

## 5. Evaluation & Analysis

### a. Metrics used
- **BLEU score** (quantitative): a standard translation evaluation metric that supports baseline vs fine-tuned comparisons.

| Model | BLEU Score |
|------|------------|
| Base OPUS-MT (MarianMT) | **50.5** |
| Fine-tuned OPUS-MT | **55.44** |

### b. Sample outputs / predictions
Sample input/output translations are provided in **`demo.ipynb`** for reproducibility and demonstration.

### c. Performance analysis and limitations
Observations:
- pretrained model already provides a strong baseline
- fine-tuning improves performance on the dataset used

Limitations:
- BLEU does not perfectly capture semantic correctness
- model struggles with:
  - very long sentences
  - rare named entities
  - domain-specific vocabulary not present in training data


In [ ]:
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.json',
 './finetuned_model/source.spm',
 './finetuned_model/target.spm',
 './finetuned_model/added_tokens.json')

In [ ]:
trainer.evaluate(test_tok)

{'eval_loss': 0.48499247431755066,
 'eval_bleu': 55.04202474921454,
 'eval_runtime': 385.7206,
 'eval_samples_per_second': 34.271,
 'eval_steps_per_second': 2.144,
 'epoch': 3.0}

In [ ]:
def translate(text):
  inputs = tokenizer(text, return_tensors="pt", truncation=True)
  inputs = {k: v.to(device) for k, v in inputs.items()}
  translated = model.generate(
      **inputs,
      max_length=128,
      num_beams=5
  )
  return tokenizer.decode(translated[0], skip_special_tokens=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/mt_project/my_translation_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/mt_project/my_translation_model/tokenizer_config.json',
 '/content/drive/MyDrive/mt_project/my_translation_model/special_tokens_map.json',
 '/content/drive/MyDrive/mt_project/my_translation_model/vocab.json',
 '/content/drive/MyDrive/mt_project/my_translation_model/source.spm',
 '/content/drive/MyDrive/mt_project/my_translation_model/target.spm',
 '/content/drive/MyDrive/mt_project/my_translation_model/added_tokens.json')

## 6. Ethical Considerations & Responsible AI

### a. Bias and fairness considerations
Potential translation bias may arise due to:
- dataset imbalance
- cultural differences in phrasing
- gendered translations in French (il/elle)

### b. Dataset limitations
Tatoeba may contain:
- mixed sentence difficulty
- crowdsourced translations which may lead to possible inconsistencies
- incomplete coverage of professional translation domains

### c. Responsible use of AI tools
- Model training performed without proprietary translation APIs
- Output should not be used for high-stakes tasks without human verification

---

## 7. Conclusion & Future Scope

### a. Summary of results
A local English → French translation system was developed by:
- preparing a parallel dataset
- fine-tuning MarianMT OPUS-MT model
- evaluating translation quality using BLEU
- deploying the model via a local Streamlit UI

### b. Possible improvements and extensions
Future scope:
- add additional language pairs (English ↔ German/Spanish)
- scale training using larger OPUS subsets
- deploy as a REST API service:
  - FastAPI backend
  - Docker-based deployment
